<a href="https://colab.research.google.com/github/RayhanLauzzadani/SC-VRP-ACO/blob/main/FP_SC_ACO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Kelompok 11
**Nama Anggota Kelompok**

1. Athaalla Rayya Genaro I - 5026221116
2. Raihan Fareliansyah - 5026221160
3. Rayhan Lauzzadani - 5026221186

#Topics
Optimasi Rute Inspeksi Kebersihan Taman Di Surabaya (Inspektur : Dinas Lingkungan Hidup)

# **Install Library**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium.plugins import PolyLineTextPath
from geopy.distance import geodesic
import random
from sklearn.cluster import KMeans
from tabulate import tabulate
# np.random.seed(30)
# random.seed(30)

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR=
# NOTEBOOK.

!pip install -q kagglehub
!pip install tabulate

import kagglehub
rayhanlauzzadani_datasetfp_path = kagglehub.dataset_download('rayhanlauzzadani/datasetsc-fp')
print(f" Dataset downloaded to: {rayhanlauzzadani_datasetfp_path}")

 Dataset downloaded to: /kaggle/input/datasetsc-fp


# **Load Dataset**

In [ ]:
# Ganti path ke file baru
df = pd.read_csv(f"{rayhanlauzzadani_datasetfp_path}/dataset_taman_surabaya_fixed.csv")

# Ambil koordinat dari baris yang mengandung "Dinas Lingkungan Hidup"
dlh_row = df[df['Nama Taman'].str.contains("Dinas Lingkungan Hidup", case=False, na=False)].iloc[0]
dlh_coord = (dlh_row['Latitude'], dlh_row['Longitude'])

# Tampilkan dataframe
print(tabulate(df, headers='keys', tablefmt='psql'))

+----+-------------------------------------------+------------+-------------+----------------+
|    | Nama Taman                                |   Latitude |   Longitude | Kategori       |
|----+-------------------------------------------+------------+-------------+----------------|
|  0 | Kantor Dinas Lingkungan Hidup             |   -7.27839 |     112.763 | pemerintah     |
|  1 | Taman. Lansia                             |   -7.271   |     112.75  | pemerintah     |
|  2 | Taman Flora                               |   -7.29427 |     112.762 | pemerintah     |
|  3 | Taman. Persahabatan                       |   -7.27673 |     112.746 | pemerintah     |
|  4 | Taman BMX Ketabang                        |   -7.26368 |     112.75  | pemerintah     |
|  5 | Taman Keputran                            |   -7.27319 |     112.744 | pemerintah     |
|  6 | Taman Ngagel                              |   -7.28844 |     112.745 | pemerintah     |
|  7 | Taman AIS Nasution                        |

# **Clustering Menjadi 3 Grup**

In [ ]:
# KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(df[['Latitude', 'Longitude']])

# Output: jumlah taman per cluster
print("Jumlah taman per cluster:")
print(df['Cluster'].value_counts().sort_index())

# Output: total taman (tanpa DLH)
total_taman_tanpa_dlh = len(df[df['Nama Taman'] != dlh_row['Nama Taman']])
print("Total taman (tanpa DLH):", total_taman_tanpa_dlh)

Jumlah taman per cluster:
Cluster
0    18
1    25
2     8
Name: count, dtype: int64
Total taman (tanpa DLH): 50


# **Distance Matrix dan Ant Colony Optimization TSP Solver**

In [ ]:
from geopy.distance import geodesic
import numpy as np

def create_distance_matrix(coords):
    n = len(coords)
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                matrix[i][j] = geodesic(coords[i], coords[j]).kilometers
    return matrix

def total_distance(path, matrix):
    return sum(matrix[path[i]][path[i+1]] for i in range(len(path)-1))

# **ACO Class**

In [ ]:
class ACO_VRPTW:
    def __init__(self, dist_matrix, service_times, depot_idx,
                 max_time_per_vehicle=8, speed=30, n_ants=10, n_iter=100,
                 alpha=1, beta=3, evaporation=0.5, Q=100):
        self.dist_matrix = dist_matrix
        self.service_times = service_times
        self.depot_idx = depot_idx
        self.n_nodes = len(dist_matrix)
        self.max_time_per_vehicle = max_time_per_vehicle
        self.speed = speed
        self.n_ants = n_ants
        self.n_iter = n_iter
        self.alpha = alpha
        self.beta = beta
        self.evaporation = evaporation
        self.Q = Q
        self.pheromone = np.ones_like(dist_matrix)
        self.heuristic = 1 / (dist_matrix + 1e-10)

    def build_solution(self):
        visited = set([self.depot_idx])
        unvisited = set(range(self.n_nodes)) - {self.depot_idx}
        routes = []
        while unvisited:
            # Mulai route baru
            route = [self.depot_idx]
            curr_idx = self.depot_idx
            waktu_kerja = 0
            while True:
                # Cari node feasible berikutnya
                feas = []
                for j in unvisited:
                    travel = self.dist_matrix[curr_idx][j] / self.speed
                    s_time = self.service_times[j]
                    pulang = self.dist_matrix[j][self.depot_idx] / self.speed
                    if waktu_kerja + travel + s_time + pulang <= self.max_time_per_vehicle:
                        feas.append(j)
                if not feas:
                    break
                probs = []
                for j in feas:
                    tau = self.pheromone[curr_idx][j] ** self.alpha
                    eta = self.heuristic[curr_idx][j] ** self.beta
                    probs.append(tau * eta)
                probs = np.array(probs)
                probs = probs / probs.sum()
                next_node = np.random.choice(feas, p=probs)
                travel = self.dist_matrix[curr_idx][next_node] / self.speed
                waktu_kerja += travel + self.service_times[next_node]
                route.append(next_node)
                visited.add(next_node)
                unvisited.remove(next_node)
                curr_idx = next_node
            # Kembali ke depot
            if route[-1] != self.depot_idx:
                waktu_kerja += self.dist_matrix[curr_idx][self.depot_idx] / self.speed
                route.append(self.depot_idx)
            routes.append(route)
        return routes

    def solve(self):
        best_routes = None
        best_cost = float('inf')
        for it in range(self.n_iter):
            all_solutions = []
            for ant in range(self.n_ants):
                routes = self.build_solution()
                total_cost = sum(
                    sum(self.dist_matrix[route[i]][route[i+1]] for i in range(len(route)-1))
                    for route in routes
                )
                all_solutions.append((routes, total_cost))
                if total_cost < best_cost:
                    best_routes = routes
                    best_cost = total_cost
            # Update pheromone
            self.pheromone *= (1 - self.evaporation)
            for routes, cost in all_solutions:
                for route in routes:
                    for i in range(len(route)-1):
                        self.pheromone[route[i]][route[i+1]] += self.Q / (cost + 1e-6)
        return best_routes


In [ ]:
import random
import numpy as np

class ACO:
    def __init__(self, n_ants, n_iterations, alpha, beta, evaporation_rate, Q,
                 service_times, kecepatan_truk_kmh=30, maks_jam_kerja=8, depot_index=0):
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.evaporation_rate = evaporation_rate
        self.Q = Q
        self.service_times = service_times
        self.kecepatan_truk_kmh = kecepatan_truk_kmh
        self.maks_jam_kerja = maks_jam_kerja
        self.depot_index = depot_index

    def _select_next_node(self, current_node, unvisited, pheromone, heuristic):
        if not unvisited:
            return None
        probs = []
        for node in unvisited:
            tau = pheromone[current_node][node] ** self.alpha
            eta = heuristic[current_node][node] ** self.beta
            probs.append(tau * eta)
        probs = np.array(probs)
        if probs.sum() == 0:
            probs = np.ones_like(probs) / len(probs)
        else:
            probs = probs / probs.sum()
        return random.choices(list(unvisited), weights=probs)[0]

    def solve(self, dist_matrix, n_vehicle=1):
        n_nodes = len(dist_matrix)
        pheromone = np.ones((n_nodes, n_nodes))
        heuristic = 1 / (dist_matrix + 1e-10)
        best_solution, best_cost, best_times = None, float('inf'), None

        for _ in range(self.n_iterations):
            all_solutions = []
            all_times = []
            for _ in range(self.n_ants):
                unvisited = set(range(n_nodes))
                unvisited.remove(self.depot_index)
                route = [self.depot_index]
                current_node = self.depot_index
                while unvisited:
                    next_node = self._select_next_node(current_node, unvisited, pheromone, heuristic)
                    if next_node is None:
                        break
                    route.append(next_node)
                    unvisited.remove(next_node)
                    current_node = next_node
                route.append(self.depot_index)
                cost = sum(dist_matrix[route[i]][route[i+1]] for i in range(len(route)-1))
                all_solutions.append((route, cost))
                if cost < best_cost:
                    best_solution = route
                    best_cost = cost
            pheromone *= (1 - self.evaporation_rate)
            for (route, cost) in all_solutions:
                for i in range(len(route) - 1):
                    pheromone[route[i]][route[i + 1]] += self.Q / (cost + 1e-6)
        return best_solution, best_cost, None


# **Proses TSP Setiap Cluster (DLH disisipkan di awal)**

In [ ]:
from datetime import datetime, timedelta

kecepatan_truk_kmh = 30
maks_jam_kerja_default = 8
maks_jam_kerja_cluster0 = 7   # UBAH DI SINI: max 7 jam utk cluster 0
start_time_str = "08:00"

def get_service_time(kategori):
    return 0.5 if kategori.strip().lower() == 'pemerintah' else 0.25

cluster_results = []

for cluster_id in sorted(df['Cluster'].unique()):
    dlh_row = df[df['Nama Taman'].str.contains("Dinas Lingkungan Hidup", case=False, na=False)].iloc[0]
    depot = pd.DataFrame({
        'Nama Taman': [dlh_row['Nama Taman']],
        'Latitude': [dlh_row['Latitude']],
        'Longitude': [dlh_row['Longitude']],
        'Kategori': [dlh_row['Kategori']],
        'Cluster': [cluster_id]
    })
    cluster_df = df[(df['Cluster'] == cluster_id) &
                    (~df['Nama Taman'].str.contains("Dinas Lingkungan Hidup", case=False, na=False))].reset_index(drop=True)
    cluster_df = pd.concat([depot, cluster_df], ignore_index=True)
    coords = cluster_df[['Latitude', 'Longitude']].values
    service_times = cluster_df['Kategori'].map(get_service_time).values
    dist_matrix = np.array([[geodesic(a, b).km for b in coords] for a in coords])

    # --- TSP (ACO single vehicle)
    tsp_solver = ACO(
        n_ants=10, n_iterations=200, alpha=1, beta=3, evaporation_rate=0.5, Q=100,
        service_times=[0]*len(cluster_df),
        kecepatan_truk_kmh=kecepatan_truk_kmh,
        maks_jam_kerja=9999,
        depot_index=0
    )
    best_path, *_ = tsp_solver.solve(dist_matrix, n_vehicle=1)
    if isinstance(best_path[0], list):
        best_path = best_path[0]
    while best_path[0] != 0:
        best_path = best_path[1:] + best_path[:1]
    if best_path[-1] != 0:
        best_path.append(0)

    curr_vehicle = 1
    time_start = datetime.strptime(start_time_str, "%H:%M")
    idx_pointer = 1
    n_nodes = len(best_path)
    batas_jam = maks_jam_kerja_cluster0 if cluster_id == 0 else maks_jam_kerja_default

    while idx_pointer < n_nodes:
        trip_idx = [0]
        total_time = 0
        last_idx = 0
        while idx_pointer < n_nodes:
            next_idx = best_path[idx_pointer]
            travel = dist_matrix[last_idx][next_idx] / kecepatan_truk_kmh
            service = 0 if next_idx == 0 else service_times[next_idx]
            balik_depot = dist_matrix[next_idx][0] / kecepatan_truk_kmh

            if (total_time + travel + service + balik_depot) > batas_jam and trip_idx[-1] != 0:
                break
            trip_idx.append(next_idx)
            total_time += travel + service
            last_idx = next_idx
            idx_pointer += 1
            if next_idx == 0:
                break
        if trip_idx[-1] != 0:
            total_time += dist_matrix[last_idx][0] / kecepatan_truk_kmh
            trip_idx.append(0)

        urut = 1
        waktu_mulai = time_start
        waktu_selesai = waktu_mulai + timedelta(hours=total_time)
        for idx in trip_idx:
            row = cluster_df.iloc[idx].copy()
            row['Cluster'] = cluster_id
            row['Kendaraan'] = curr_vehicle
            row['Urutan'] = urut
            row['Waktu Mulai'] = waktu_mulai.strftime("%H:%M")
            row['Waktu Selesai'] = waktu_selesai.strftime("%H:%M")
            cluster_results.append(row)
            urut += 1
        curr_vehicle += 1
        time_start = waktu_selesai

df_rute_final = pd.DataFrame(cluster_results).reset_index(drop=True)

for cl in sorted(df_rute_final['Cluster'].unique()):
    n_vehicle = df_rute_final[df_rute_final['Cluster'] == cl]['Kendaraan'].nunique()
    print(f"Cluster {cl}: {n_vehicle} kendaraan digunakan.")

print("\nTabulate preview:")
print(tabulate(
    df_rute_final[['Cluster', 'Kendaraan', 'Urutan', 'Nama Taman', 'Kategori', 'Waktu Mulai', 'Waktu Selesai']],
    headers='keys', tablefmt='psql', showindex=False
))


Cluster 0: 2 kendaraan digunakan.
Cluster 1: 2 kendaraan digunakan.
Cluster 2: 1 kendaraan digunakan.

Tabulate preview:
+-----------+-------------+----------+-------------------------------------------+----------------+---------------+-----------------+
|   Cluster |   Kendaraan |   Urutan | Nama Taman                                | Kategori       | Waktu Mulai   | Waktu Selesai   |
|-----------+-------------+----------+-------------------------------------------+----------------+---------------+-----------------|
|         0 |           1 |        1 | Kantor Dinas Lingkungan Hidup             | pemerintah     | 08:00         | 14:58           |
|         0 |           1 |        2 | Taman Ngagel                              | pemerintah     | 08:00         | 14:58           |
|         0 |           1 |        3 | Taman Ujung Galuh                         | non-pemerintah | 08:00         | 14:58           |
|         0 |           1 |        4 | Taman Bungkul                       

# Mencari Seed untuk optimasi rute yang paling optimal

In [ ]:
from datetime import timedelta, datetime
import numpy as np
import pandas as pd
import random
from geopy.distance import geodesic

# --- DEFINISI KONFIGURASI ---
kecepatan_truk_kmh = 30
start_time_str = "08:00"
vehicle_per_cluster = {0: 2, 1: 3, 2: 1}
cluster_max_hours = {0: 7, 1: 8, 2: 8}  # cluster 0 khusus max 7 jam

# --- HELPER FUNCTION ---
def get_inspeksi(kategori):
    return 0.5 if kategori.strip().lower() == 'pemerintah' else 0.25

def create_distance_matrix(coords):
    n = len(coords)
    matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                matrix[i][j] = geodesic(coords[i], coords[j]).kilometers
    return matrix

def get_dlh_df(cluster_id):
    return pd.DataFrame({
        'Nama Taman': [dlh_row['Nama Taman']],
        'Latitude': [dlh_row['Latitude']],
        'Longitude': [dlh_row['Longitude']],
        'Kategori': [dlh_row['Kategori']],
        'Cluster': [cluster_id],
        'ServiceTime': [get_inspeksi(dlh_row['Kategori'])]
    })

# --- GRID SEARCH SEED ---
best_total_time = float('inf')
best_seed = None
best_rute_final = None
best_hasil_per_cluster = None

for seed in range(1, 102):
    np.random.seed(seed)
    random.seed(seed)
    cluster_results = []

    for cluster_id in sorted(df['Cluster'].unique()):
        cluster_df = df[(df['Cluster'] == cluster_id) & (~df['Nama Taman'].str.contains("Dinas Lingkungan Hidup", case=False))].reset_index(drop=True)
        dlh_df = get_dlh_df(cluster_id)
        sisa_idx = list(cluster_df.index)
        vehicle_num = 1
        time_start = datetime.strptime(start_time_str, "%H:%M")
        max_jam = cluster_max_hours.get(cluster_id, 8)

        while sisa_idx:
            route = [0]
            waktu_kerja = 0
            curr_idx = 0
            sub_df = pd.concat([dlh_df, cluster_df.loc[sisa_idx]], ignore_index=True)
            coords = sub_df[['Latitude', 'Longitude']].values
            dist_matrix = create_distance_matrix(coords)
            visited_local = []

            while True:
                pilihan = []
                for idx in range(1, len(sub_df)):
                    if idx-1 in visited_local:
                        continue
                    travel = dist_matrix[curr_idx][idx] / kecepatan_truk_kmh
                    service = sub_df.iloc[idx]['ServiceTime']
                    pulang = dist_matrix[idx][0] / kecepatan_truk_kmh
                    if waktu_kerja + travel + service + pulang > max_jam:
                        continue
                    pilihan.append((idx, travel, service))
                if not pilihan:
                    break
                pilihan.sort(key=lambda x: x[1] + 0.01 * random.random())
                next_idx = pilihan[0][0]
                waktu_kerja += pilihan[0][1] + pilihan[0][2]
                curr_idx = next_idx
                route.append(curr_idx)
                visited_local.append(curr_idx - 1)

            route.append(0)
            waktu_kerja += dist_matrix[curr_idx][0] / kecepatan_truk_kmh

            # Setelah kalkulasi waktu_kerja
            if np.isnan(waktu_kerja) or len(route) <= 2:
                break  # Skip trip jika gagal dapat titik

            # Simpan hasil trip
            route_result = sub_df.iloc[route].copy().reset_index(drop=True)
            route_result['Urutan'] = range(1, len(route_result) + 1)
            route_result['Kendaraan'] = vehicle_num
            route_result['Cluster'] = cluster_id
            route_result['Waktu Mulai'] = time_start.strftime('%H:%M')
            route_result['Waktu Selesai'] = (time_start + timedelta(hours=waktu_kerja)).strftime('%H:%M')
            route_result['Total Jam'] = round(waktu_kerja, 2)
            cluster_results.append(route_result)

            real_idx_visited = [sisa_idx[v] for v in visited_local]
            sisa_idx = [x for x in sisa_idx if x not in real_idx_visited]

            time_start += timedelta(hours=waktu_kerja)
            vehicle_num += 1
            if vehicle_num > vehicle_per_cluster.get(cluster_id, 100):
                break

    if len(cluster_results) == 0:
        continue
    df_rute_final = pd.concat(cluster_results, ignore_index=True)

    # Evaluasi maksimum waktu tempuh antar kendaraan
    waktu_terlama = df_rute_final.groupby(['Cluster', 'Kendaraan'])['Total Jam'].max().max()
    if waktu_terlama < best_total_time:
        best_total_time = waktu_terlama
        best_seed = seed
        best_rute_final = df_rute_final.copy()

# --- OUTPUT ---
print(f"\n✅ Seed terbaik: {best_seed} dengan waktu tempuh maksimal: {round(best_total_time, 2)} jam")
pd.set_option('display.max_columns', None)
display(best_rute_final)



✅ Seed terbaik: None dengan waktu tempuh maksimal: inf jam


None

# **Visualisasi Pembagian Cluster**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
colors = ['green', 'red', 'blue', 'orange', 'purple', 'cyan']
for cluster_id, color in zip(sorted(df['Cluster'].unique()), colors):
    cluster_df = df[df['Cluster'] == cluster_id]
    plt.scatter(cluster_df['Longitude'], cluster_df['Latitude'], label=f'Cluster {cluster_id}', color=color, s=60)

plt.scatter(dlh_coord[1], dlh_coord[0], c='black', marker='X', s=200, label='DLH (Depot)')
plt.title('Sebaran Taman dan Cluster (Hasil KMeans)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.grid(True)
plt.show()


# **Visualisasi Rute TSP Optimal + DLH**

In [ ]:
import matplotlib.pyplot as plt
import itertools

plt.figure(figsize=(10, 8))
# Banyak warna, supaya cukup untuk semua kendaraan!
color_list = ['green', 'red', 'blue', 'orange', 'purple', 'cyan', 'brown', 'magenta', 'olive', 'grey']
color_cycle = itertools.cycle(color_list)

for cluster_id in sorted(df_rute_final['Cluster'].unique()):
    cluster_rute = df_rute_final[df_rute_final['Cluster'] == cluster_id]
    for kendaraan in sorted(cluster_rute['Kendaraan'].unique()):
        trip = cluster_rute[cluster_rute['Kendaraan'] == kendaraan].sort_values('Urutan')
        trip = pd.concat([trip, trip.iloc[[0]]], ignore_index=True)
        warna = next(color_cycle)
        plt.plot(trip['Longitude'], trip['Latitude'], '-o',
                 label=f'Cluster {cluster_id} - Kendaraan {kendaraan}',
                 alpha=0.75, color=warna)
        plt.scatter(trip.iloc[0]['Longitude'], trip.iloc[0]['Latitude'],
                    c='black', marker='X', s=150)

plt.title('Rute VRPTW (Setiap Kendaraan Berbeda Warna)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.grid(True)
plt.show()

# **Visualisasi Folium**

In [ ]:
import folium
from folium.plugins import PolyLineTextPath

# Daftar warna yang didukung Folium
folium_vehicle_colors = [
    'red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
    'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
    'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray'
]

# Ganti 'Vehicle' ke 'Kendaraan' jika pakai kolom 'Kendaraan'
kolom_vehicle = 'Kendaraan' if 'Kendaraan' in df_rute_final.columns else 'Vehicle'

# Mapping (Cluster, Vehicle) ke warna
vehicle_ids = (
    df_rute_final[['Cluster', kolom_vehicle]]
    .drop_duplicates()
    .sort_values(['Cluster', kolom_vehicle])
    .reset_index(drop=True)
)
vehicle_ids['color'] = [folium_vehicle_colors[i % len(folium_vehicle_colors)] for i in range(len(vehicle_ids))]
vehicle_color_map = {(row['Cluster'], row[kolom_vehicle]): row['color'] for _, row in vehicle_ids.iterrows()}

# Peta awal di depot
m = folium.Map(location=dlh_coord, zoom_start=13)

for cluster_id in sorted(df_rute_final['Cluster'].unique()):
    r = df_rute_final[df_rute_final['Cluster'] == cluster_id]
    for v in sorted(r[kolom_vehicle].unique()):
        trip = r[r[kolom_vehicle]==v].sort_values('Urutan')
        # Tutup loop ke depot jika belum
        if not (
            np.isclose(trip.iloc[0]['Longitude'], trip.iloc[-1]['Longitude']) and
            np.isclose(trip.iloc[0]['Latitude'], trip.iloc[-1]['Latitude'])
        ):
            trip = pd.concat([trip, trip.iloc[0:1]], ignore_index=True)
        points = list(zip(trip['Latitude'], trip['Longitude']))
        color = vehicle_color_map[(cluster_id, v)]
        line = folium.PolyLine(points, color=color, weight=4, opacity=0.7).add_to(m)
        PolyLineTextPath(line, '➤   ', repeat=True, offset=6, attributes={'fill': color}).add_to(m)
        for _, row in trip.iterrows():
            folium.Marker(
                location=[row['Latitude'], row['Longitude']],
                popup=f"{row['Nama Taman']} (Cluster {cluster_id}, Vehicle {v}, Urutan {row['Urutan']})",
                icon=folium.Icon(color=color)
            ).add_to(m)

# Marker untuk depot
folium.Marker(
    location=dlh_coord,
    popup="DLH (Depot)",
    icon=folium.Icon(color="black", icon="home")
).add_to(m)

m

# **Detail Jarak Antar Taman per Cluster (Urutan Rute ACO)**


In [ ]:
from geopy.distance import geodesic
from tabulate import tabulate

kolom_vehicle = 'Kendaraan' if 'Kendaraan' in df_rute_final.columns else 'Vehicle'

for cluster_id in sorted(df_rute_final['Cluster'].unique()):
    print(f"\n{'='*30}\nRUTE CLUSTER {cluster_id}\n{'='*30}")
    r = df_rute_final[df_rute_final['Cluster'] == cluster_id].sort_values([kolom_vehicle, 'Urutan']).reset_index(drop=True)
    for v in r[kolom_vehicle].unique():
        trip = r[r[kolom_vehicle] == v].sort_values('Urutan').reset_index(drop=True)
        trip = pd.concat([trip, trip.iloc[0:1]], ignore_index=True)
        output_rows = []
        for i in range(len(trip) - 1):
            a, b = trip.iloc[i], trip.iloc[i+1]
            jarak = geodesic((a['Latitude'], a['Longitude']), (b['Latitude'], b['Longitude'])).kilometers
            # Kategori fallback ke 'Status' jika tidak ada
            status_a = a['Kategori'] if 'Kategori' in a else a.get('Status', '-')
            status_b = b['Kategori'] if 'Kategori' in b else b.get('Status', '-')
            # Skip baris depot ke depot jika jaraknya 0 (atau sangat kecil, toleransi <0.01 km)
            if (a['Nama Taman'] == b['Nama Taman']) and (jarak < 0.01):
                continue
            output_rows.append({
                'Dari': f"{a['Nama Taman']} ({status_a})",
                'Ke': f"{b['Nama Taman']} ({status_b})",
                'Jarak (km)': f"{jarak:.2f}"
            })
        print(f"\n--- {kolom_vehicle} {v} ---")
        print(tabulate(output_rows, headers='keys', tablefmt='psql', showindex=False))

# **Total Jarak & Waktu Tempuh Truk per Cluster (Fixed Speed 30 km/jam)**

In [ ]:
from geopy.distance import geodesic
from tabulate import tabulate

kecepatan_truk_kmh = 30  # <--- TAMBAHKAN INI JIKA BELUM ADA

hasil_per_cluster = []
kolom_vehicle = 'Kendaraan' if 'Kendaraan' in df_rute_final.columns else 'Vehicle'

for cluster_id in sorted(df_rute_final['Cluster'].unique()):
    r = df_rute_final[df_rute_final['Cluster'] == cluster_id]
    for v in r[kolom_vehicle].unique():
        trip = r[r[kolom_vehicle] == v].sort_values('Urutan')
        trip = pd.concat([trip, trip.iloc[0:1]], ignore_index=True)
        total_jarak = 0
        total_inspeksi_jam = 0
        n_taman = len(trip) - 2  # exclude depot awal & akhir

        for i in range(len(trip) - 1):
            coord_a = (trip.iloc[i]['Latitude'], trip.iloc[i]['Longitude'])
            coord_b = (trip.iloc[i+1]['Latitude'], trip.iloc[i+1]['Longitude'])
            total_jarak += geodesic(coord_a, coord_b).kilometers
            # Waktu inspeksi hanya jika bukan depot
            if i > 0 and i < len(trip) - 2 and 'ServiceTime' in trip.columns:
                total_inspeksi_jam += trip.iloc[i]['ServiceTime']
            elif i > 0 and i < len(trip) - 2:
                total_inspeksi_jam += 0.5

        waktu_jalan_jam = total_jarak / kecepatan_truk_kmh
        waktu_total_jam = waktu_jalan_jam + total_inspeksi_jam

        jam = int(waktu_total_jam)
        menit = int((waktu_total_jam - jam) * 60)
        waktu_tempuh_format = f"{jam} jam {menit} menit" if jam > 0 else f"{menit} menit"

        hasil_per_cluster.append({
            'Cluster': cluster_id,
            kolom_vehicle: v,
            'Jumlah Taman': n_taman,
            'Total Jarak (km)': round(total_jarak, 2),
            'Waktu Tempuh': waktu_tempuh_format
        })

hasil_df = pd.DataFrame(hasil_per_cluster)
print(tabulate(hasil_df, headers='keys', tablefmt='psql', showindex=False))